In [ ]:
import math
from sklearn.neighbors import KNeighborsClassifier
import keras
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KDTree
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
import cv2
import numpy as np
from matplotlib import pyplot as plt
import operator
from keras import *
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from keras.layers import Activation, LeakyReLU
from tensorflow.keras.activations import sigmoid
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from keras.backend import abs, sum


In [ ]:
train_processing = keras.preprocessing.image.ImageDataGenerator(rotation_range=45, 
                                                                width_shift_range=0.1, 
                                                                height_shift_range=0.1, 
                                                                zoom_range=0.1, 
                                                                preprocessing_function=mobilenet_v2.preprocess_input)###resnet_v2 mobilenet_v2 vgg16
train_generator= train_processing.flow_from_directory( 
    directory='mask_NEW_test_folders',
    target_size=(224,224), #scelta standard
    color_mode= 'rgb', #dobbiamo mettere stessi parametri della rete di partenza
    class_mode = 'categorical',
    batch_size = 16,
    shuffle = True,
    seed = 1
)

In [ ]:
test_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=mobilenet_v2.preprocess_input)###resnet_v2 mobilenet_v2 vgg16
test_generator= test_processing.flow_from_directory(
    directory='mask_NEW_train_folders',
    target_size=(224,224), #scelta standard
    color_mode= 'rgb', #dobbiamo mettere stessi parametri della rete di partenza
    class_mode = 'categorical',
    batch_size = 16,
    shuffle = True,
    seed = 1
)

In [ ]:
base_net = mobilenet_v2.MobileNetV2(input_shape=(224,224,3), weights ='imagenet', include_top=False, pooling='avg')

In [ ]:
base_net = resnet_v2.ResNet101V2(input_shape=(224,224,3), weights ='imagenet', include_top=False, pooling='avg')

In [ ]:
base_net = vgg16.VGG16(input_shape=(224,224,3), weights ='imagenet', include_top=False, pooling='avg')

In [ ]:
base_net.summary()

In [ ]:
for layers in base_net.layers:
    layers.trainable = False

In [ ]:
x = base_net.output #base_net.get_layer('fc1').output

x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(128, activation = 'relu', kernel_regularizer=regularizers.l2(0.0005))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)

pred = keras.layers.Dense(102, activation='softmax')(x) 


In [ ]:
net = keras.Model(inputs=base_net.input, outputs=pred)

In [ ]:
net.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.Adam(), #keras.optimizers.RMSprop() #keras.optimizers.Adam() #sgd
            metrics=['accuracy'])

In [ ]:
history = net.fit(train_generator, batch_size=64, epochs=30, verbose=1, validation_data=test_generator) ####30

In [ ]:
net.save('mod.h5')

In [ ]:
def plot_history(network_history):
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(x_plot, network_history.history['loss'])
    plt.plot(x_plot, network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(x_plot, network_history.history['accuracy'])
    plt.plot(x_plot, network_history.history['val_accuracy'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()

In [ ]:
epochs = 30
x_plot = list(range(1,epochs+1))
plot_history(network)